In [1]:
import ssl
import certifi
import json
import csv
import psycopg2
from dotenv import load_dotenv
import os
from urllib.request import urlopen 
import requests


In [2]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

In [3]:
def get_jsonparsed_data(url):
    context = ssl.create_default_context(cafile=certifi.where())
    response = urlopen(url, context=context)
    data = response.read().decode("utf-8")
    return json.loads(data)
    

In [19]:
def fetch_indian_companies(api_key):
    url = f"https://financialmodelingprep.com/api/v3/stock/list?apikey={api_key}"
    stock_list = get_jsonparsed_data(url)
    
    indian_exchanges = ["NSE", "BSE"]
    indian_companies = [
        stock for stock in stock_list if stock.get("exchangeShortName") in indian_exchanges
    ]
    
    print(f"Total Indian Companies Found: {len(indian_companies)}")
    
    for company in indian_companies[:5]:
        print(f"Symbol: {company['symbol']}, Name: {company['name']}, Exchange: {company['exchangeShortName']}, Type: {company.get('type')}")
    
    return indian_companies

In [18]:
def save_to_csv(data, filename):
    filtered_data = [
        {
            "symbol": item.get("symbol"),
            "name": item.get("name"),
            "exchange": item.get("exchange"),
            "exchangeShortName": item.get("exchangeShortName"),
            "type": item.get("type"),
        }
        for item in data
    ]
    
    if filtered_data:
        with open(filename, "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=["symbol", "name", "exchange", "exchangeShortName", "type"])
            writer.writeheader()
            writer.writerows(filtered_data)
        print(f"Data saved to {filename}")

In [20]:
def push_to_postgres(data, db_config):
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(**db_config)
        cur = conn.cursor()
        
        # Create table if not exists
        create_table_query = """
        CREATE TABLE IF NOT EXISTS indian_companies (
            id SERIAL PRIMARY KEY,
            symbol TEXT,
            name TEXT,
            exchange TEXT,
            exchangeShortName TEXT,
            type TEXT 
        );
        """
        cur.execute(create_table_query)
        
        # Insert data (exclude price field)
        for company in data:
            insert_query = """
            INSERT INTO indian_companies (symbol, name, exchange, exchangeShortName, type)
            VALUES (%s, %s, %s, %s, %s)
            """
            cur.execute(insert_query, (
                company.get("symbol"),
                company.get("name"),
                company.get("exchange"),
                company.get("exchangeShortName"),
                company.get("type"),
            ))
        
        # Commit changes and close connection
        conn.commit()
        cur.close()
        conn.close()
        print("Data successfully pushed to PostgreSQL (price excluded).")
    except Exception as e:
        print(f"Error pushing data to PostgreSQL: {e}")


In [23]:
# PostgreSQL configuration
db_config = {
    "host": "localhost",
    "database": "chain8_fmp",
    "user": "chain8_fmp",
    "password": "chain8_fmp",
    "port": "5433"
}

In [22]:
# Fetch data
indian_companies = fetch_indian_companies(API_KEY)

# Save to CSV
csv_filename = "indian_companies_fmp.csv"
save_to_csv(indian_companies, csv_filename)



Total Indian Companies Found: 6440
Symbol: GOKAKTEX.BO, Name: Gokak Textiles Limited, Exchange: BSE, Type: stock
Symbol: IDEA.BO, Name: Vodafone Idea Limited, Exchange: BSE, Type: stock
Symbol: AJMERA.BO, Name: Ajmera Realty & Infra India Limited, Exchange: BSE, Type: stock
Symbol: RUDRA.BO, Name: Rudra Global Infra Products Limited, Exchange: BSE, Type: stock
Symbol: SPICEJET.BO, Name: SpiceJet Limited, Exchange: BSE, Type: stock
Data saved to indian_companies_fmp.csv
Error pushing data to PostgreSQL: invalid integer value "5433:5432" for connection option "port"



In [24]:
# Push to PostgreSQL
push_to_postgres(indian_companies, db_config)

Data successfully pushed to PostgreSQL (price excluded).
